In [32]:
from tensorflow.keras import models,layers,Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [86]:
simpleConv = models.Sequential([
    layers.Conv2D(16,(3,3),activation='relu',input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32,(3,3),activation='relu'),
])

In [87]:
simpleConv.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 11, 11, 32)        4640      
Total params: 4,800
Trainable params: 4,800
Non-trainable params: 0
_________________________________________________________________


In [88]:
layer = layers.Flatten()(simpleConv.output)
layer = layers.Dense(64, activation='relu')(layer)
layer = layers.Dense(10, activation='softmax')(layer)

In [89]:
totalModel = Model(simpleConv.input,layer)
totalModel.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16_input (InputLayer) [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 11, 11, 32)        4640      
_________________________________________________________________
flatten_2 (Flatten)          (None, 3872)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                247872    
_________________________________________________________________
dense_12 (Dense)             (None, 10)              

In [90]:
totalModel.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [24]:
import pandas as pd
import numpy as np

In [22]:
train_data = pd.read_csv('digit-recognizer/train.csv')

In [23]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
train_labels = train_data['label'].to_numpy()

In [85]:
train_images = np.zeros((train_labels.shape[0],28,28))
for index, row in train_data.iterrows():
    npRow = row.to_numpy()
    npRow = npRow[1:]
    image = np.reshape(npRow, (28,28))
    train_images[index]=image
train_images = train_images[...,np.newaxis]
train_images/=255

In [91]:
totalModel.fit(train_images, train_labels, epochs=20, batch_size=128, validation_split=0.2, callbacks=[ModelCheckpoint('convnet',monitor='val_acc',save_best_only=True)])

Epoch 1/20
263/263 [==============================] - 7s 26ms/step - loss: 0.3184 - acc: 0.9084 - val_loss: 0.1006 - val_acc: 0.9692
Epoch 2/20
263/263 [==============================] - 7s 28ms/step - loss: 0.0794 - acc: 0.9772 - val_loss: 0.0652 - val_acc: 0.9796
Epoch 3/20
263/263 [==============================] - 7s 25ms/step - loss: 0.0542 - acc: 0.9843 - val_loss: 0.0541 - val_acc: 0.9819
Epoch 4/20
263/263 [==============================] - 8s 31ms/step - loss: 0.0424 - acc: 0.9877 - val_loss: 0.0504 - val_acc: 0.9846
Epoch 5/20
263/263 [==============================] - 7s 26ms/step - loss: 0.0328 - acc: 0.9899 - val_loss: 0.0460 - val_acc: 0.9857
Epoch 6/20
263/263 [==============================] - 6s 23ms/step - loss: 0.0246 - acc: 0.9927 - val_loss: 0.0486 - val_acc: 0.9855
Epoch 7/20
263/263 [==============================] - 6s 25ms/step - loss: 0.0184 - acc: 0.9944 - val_loss: 0.0529 - val_acc: 0.9851
Epoch 8/20
263/263 [==============================] - 6s 24ms/step - 

In [92]:
totalModel = models.load_model('convnet')

In [55]:
test_data = pd.read_csv('digit-recognizer/test.csv')

In [93]:
test_images = np.zeros((len(test_data),28,28))
for index, row in test_data.iterrows():
    npRow = row.to_numpy()
    image = np.reshape(npRow, (28,28))
    test_images[index]=image
test_images = test_images[...,np.newaxis]
test_images/=255

In [94]:
results = totalModel.predict(test_images)

In [95]:
output = []
for i in range(len(results)):
    output.append([i+1,np.argmax(results[i])])

In [96]:
output = pd.DataFrame(output,columns=['ImageID','Label'])

In [97]:
output.head()

,ImageID,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [98]:
output.to_csv('output.csv',index=False)